In [3]:
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import Table, TableStyle
from PyPDF2 import PdfMerger
import datetime
import os

# Function to load order data from the CSV file
def load_orders(file_path):
    return pd.read_csv(file_path)

# Function to generate individual PDF invoices
def generate_invoice(order, output_dir="invoices"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Extract order details
    order_id = order['Order ID']
    customer_name = order['Customer Name']
    product_name = order['Product Name']
    quantity = order['Quantity']
    unit_price = order['Unit Price']
    total_amount = quantity * unit_price
    date_of_purchase = datetime.date.today().strftime("%Y-%m-%d")

    # PDF file path
    pdf_file = os.path.join(output_dir, f"Invoice_{order_id}.pdf")

    # Create the PDF
    c = canvas.Canvas(pdf_file, pagesize=A4)
    c.setFont("Helvetica-Bold", 14)
    c.drawString(230, 800, f"Invoice Number: {order_id}")
    c.setFont("Helvetica", 12)
    c.drawString(100, 780, f"Date of Purchase: {date_of_purchase}")

    # Table data
    data = [
        ["Field", "Details"],
        ["Customer Name", customer_name],
        ["Product Name", product_name],
        ["Quantity", quantity],
        ["Unit Price", f"${unit_price:.2f}"],
        ["Total Amount", f"${total_amount:.2f}"]
    ]

    # Create the table
    table = Table(data, colWidths=[2.5 * inch, 3.5 * inch])

    # Table style
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),  # Header background
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),  # Header text color
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center align
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),  # Header font
        ('FONTSIZE', (0, 0), (-1, 0), 12),  # Header font size
        ('FONTSIZE', (0, 1), (-1, -1), 10),  # Content font size
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),  # Content background
        ('GRID', (0, 0), (-1, -1), 1, colors.black),  # Grid lines
    ]))

    # Position the table on the PDF
    table.wrapOn(c, 400, 200)
    table.drawOn(c, 100, 600)

    # Save the PDF
    c.save()
    return pdf_file

# Function to merge all generated PDF invoices into a single file
def merge_invoices(invoice_files, output_file="Merged_Invoices.pdf"):
    merger = PdfMerger()
    for pdf in invoice_files:
        merger.append(pdf)
    merger.write(output_file)
    merger.close()

# Main function to process the data and generate invoices
def main():
    # Load order data
    orders = load_orders("orders.csv")

    # Generate individual invoices and store file paths
    invoice_files = []
    for _, order in orders.iterrows():
        pdf_file = generate_invoice(order)
        invoice_files.append(pdf_file)

    # Merge all invoices into a single file
    merge_invoices(invoice_files)
    print("All invoices have been generated and merged into 'Merged_Invoices.pdf'.")

# Execute the script
if __name__ == "__main__":
    main()


All invoices have been generated and merged into 'Merged_Invoices.pdf'.


In [6]:
import pandas as pd
from fpdf import FPDF
from PyPDF2 import PdfMerger
import datetime
import os

# Load Order Data
def load_orders(file_path):
    return pd.read_csv(file_path)

# Class to create PDF invoices using FPDF
class InvoicePDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Invoice', align='C', ln=True)

    def add_invoice_details(self, order):
        # Add order details to the PDF
        self.set_font('Arial', '', 12)
        self.ln(10)  # Line break
        self.cell(0, 10, f"Invoice Number: {order['Order ID']}", ln=True)
        self.cell(0, 10, f"Date of Purchase: {datetime.date.today().strftime('%Y-%m-%d')}", ln=True)
        self.cell(0, 10, f"Customer Name: {order['Customer Name']}", ln=True)
        self.cell(0, 10, f"Product Name: {order['Product Name']}", ln=True)
        self.cell(0, 10, f"Quantity: {order['Quantity']}", ln=True)
        self.cell(0, 10, f"Unit Price: ${order['Unit Price']:.2f}", ln=True)
        self.cell(0, 10, f"Total Amount: ${order['Quantity'] * order['Unit Price']:.2f}", ln=True)

# Function to generate individual invoices
def generate_invoice(order, output_dir="invoices"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Create PDF instance
    pdf = InvoicePDF()
    pdf.add_page()
    pdf.add_invoice_details(order)

    # Save the PDF
    pdf_file = os.path.join(output_dir, f"Invoice_{order['Order ID']}.pdf")
    pdf.output(pdf_file)
    return pdf_file

# Function to merge invoices into a single PDF
def merge_invoices(invoice_files, output_file="Merged_Invoices.pdf"):
    merger = PdfMerger()
    for pdf in invoice_files:
        merger.append(pdf)
    merger.write(output_file)
    merger.close()

# Main function
def main():
    # Load orders
    orders = load_orders("orders.csv")

    # Generate individual invoices
    invoice_files = []
    for _, order in orders.iterrows():
        pdf_file = generate_invoice(order)
        invoice_files.append(pdf_file)

    # Merge all invoices
    merge_invoices(invoice_files)
    print("All invoices have been merged into Merged_Invoices.pdf")

if __name__ == "__main__":
    main()


All invoices have been merged into Merged_Invoices.pdf
